## this is from the hugging face implemenation of the Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers

<link>https://huggingface.co/blog/fine-tune-whisper</link>

#### import the required modules

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
#common stuff
model_version = "openai/whisper-small"
model_lang = "English"
model_task="transcribe"
dataset_name ="Tarakeshwaran/Whisper-train-data"

### load the dataset

In [3]:
from datasets import load_dataset, DatasetDict

whisper_data = DatasetDict()

whisper_data["train"] = load_dataset(dataset_name,split="train")
whisper_data["test"] = load_dataset(dataset_name,split="test")

print(whisper_data)

DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'start', 'end'],
        num_rows: 80
    })
    test: Dataset({
        features: ['audio', 'text', 'start', 'end'],
        num_rows: 20
    })
})


In [4]:
#remove unwanted columns
whisper_data = whisper_data.remove_columns([ 'start', 'end'])

## Prepare Feature Extractor, Tokenizer and Data

In [5]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_version)

In [6]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_version, language=model_lang, task=model_task)

In [7]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_version, language=model_lang, task=model_task)

In [8]:
input_str = whisper_data["train"][0]["text"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 in alchemy it's called the soul of the world
Decoded w/ special:    <|startoftranscript|><|en|><|transcribe|><|notimestamps|>in alchemy it's called the soul of the world<|endoftext|>
Decoded w/out special: in alchemy it's called the soul of the world
Are equal:             True


In [9]:
print(whisper_data["train"][0])

{'audio': {'path': 'sample-005811.mp3', 'array': array([-3.10192730e-25,  8.27180613e-25, -4.65289095e-24, ...,
       -1.07697597e-05,  2.15554501e-05, -2.88033916e-05]), 'sampling_rate': 16000}, 'text': "in alchemy it's called the soul of the world"}


In [10]:
from datasets import Audio

whisper_data = whisper_data.cast_column("audio", Audio(sampling_rate=16000))

In [11]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [12]:
whisper_data = whisper_data.map(
    prepare_dataset,
    remove_columns=whisper_data.column_names["train"],
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/80 [00:00<?, ? examples/s]

NameError: name 'feature_extractor' is not defined